# Configure Flavor path

Here we will give the user the option between:
 - the Exasol <a href="https://github.com/exasol/script-languages-release" target="_blank" rel="noopener">Script-Languages-Release</a> Github repository which provides the Script-Languages-Container flavor used to build the builtin container of the Exasol DB or to build customized container
 - to use a custom flavor path

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).

## Setup

### Open Secure Configuration Storage


In [2]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

# Load UI functions
Let's import some other UI functions that may be used in this notebook.

In [22]:
%run ./utils/slc_ui.ipynb

Output()

# Configure the Flavor path
### Choose the flavor source

In [23]:
display(get_slc_source_selection_ui(ai_lab_config))

Box(children=(Box(children=(Label(value='Flavor choice', layout=Layout(border_bottom='solid 1px', border_left=…

### Use existing script-languages-repository
If the user has chosen to use an existing script-languages-repository path we simply let him choose the path.

In [24]:
display(get_existing_slc_ui(ai_lab_config))

ParentPathError: /home/jupyter/script_languages_release is not a part of /home/jupyter/notebooks

### Clone the Script-Languages-Release repository
If the user wants to use the Exasol script-languages-repository we need first to choose the root path where the repository should be stored.

In [8]:
display(get_slc_target_dir_ui(ai_lab_config))

Box(children=(Box(children=(Label(value='Target Directory', layout=Layout(border_bottom='solid 1px', border_le…

#### Now we can clone the repository

In [9]:
from pathlib import Path
if clone_slc_repo(ai_lab_config):
    from git import Repo
    
    slc_dir = Path(ai_lab_config.get(slc_target_dir_store_key))
    if not slc_dir.is_dir():
        print (f"Cloning into {slc_dir}...")
        repo = Repo.clone_from("https://github.com/exasol/script-languages-release", slc_dir)
        print ("Fetching submodules...")
        repo.submodule_update(recursive=True)
    else:
        print(f"Directory '{slc_dir}' already exists. Skipping cloning....")
print("Ready")

Cloning into /home/jupyter/script_languages_release...
Fetching submodules...
Ready


#### Now we verify that the required flavor for the tutorial is present.

In [18]:
flavor_base_dir = Path(slc_dir) / "flavors"
flavor_dirs =  [flavor_dir.name for flavor_dir in flavor_base_dir.iterdir() if flavor_dir.is_dir()]
required_flavor = "template-Exasol-all-python-3.10"
filtered_flavors = [flavor for flavor in flavor_dirs if str(flavor) == required_flavor]
if len(filtered_flavors) == 0:
    popup_message(f"The script-languages repository at '{slc_dir}' does not contain flavor '{required_flavor}'")
assert len(filtered_flavors) == 1
ai_lab_config.save(slc_source_flavor_store_key, str(required_flavor))